# Clustering mit Scikit-Learn
In diesem Tutorial werden wir uns mit dem Thema Clustering beschäftigen. Clustering ist eine unüberwachte Lernmethode, die es ermöglicht, Daten in Gruppen zu unterteilen, ohne dass vorherige Labels oder Kategorien bekannt sind. Wir werden die Bibliothek Scikit-Learn verwenden, um verschiedene Clustering-Algorithmen zu implementieren und zu visualisieren.

In diesem Beispiel werden wir mit der Sammlung von State-of-the-Union-Ansprachen arbeiten. Beachte, dass diese uns erlaubt, auf komplexeres Preprocessing zu verzichten, weil sie auf englisch ist. Für einen deutschen Text ist ein Vorgehen wie im Tutorial zu Topic Modeling empfehlenswert um z.B. Wortformen zu normalisieren. Auch ein englischer Text könnte aber von einem ausführlicheren Preprocessing profitieren. In diesem Tutorial wurde zu Gunsten der Kürze darauf verzichtet.

In [ ]:
# Einlesen des Textes
with open("state_of_the_union.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [ ]:
# Wir teilen den Text in Absätze auf, sehr rudimentär
texts = text.split("\n\n")

print(texts[56])

Die Bibliothek [scikit-learn](https://scikit-learn.org/stable/) ist eine der am häufigsten verwendeten Bibliotheken für maschinelles Lernen in Python. Sie bietet eine Vielzahl von Algorithmen und Tools für verschiedene Aufgaben, einschließlich Klassifikation, Regression und Clustering. In diesem Tutorial konzentrieren wir uns auf die Clustering-Algorithmen in Scikit-Learn.

In [ ]:
%pip install -U -q scikit-learn numpy

Um die Texte zu clustern, müssen wir sie in Vektoren (Embeddings) umwandeln. Wir könnten dies auf verschiedene Arten tun, z.B. auch mithilfe eines Sprachmodells aus einem Framework wie Flair oder Spacy. Wir behelfen uns hier mit einer einfacheren Methode [tf-idf](https://de.wikipedia.org/wiki/Tf-idf), welche Texte durch relative Worthäufigkeit repräsentiert. Diese Methode ist einfach zu implementieren und liefert in vielen Fällen gute Ergebnisse. Wir verwenden die Klasse `TfidfVectorizer` aus Scikit-Learn, um die Texte in Vektoren umzuwandeln.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vorbereiten der Vektorisierung, Stoppwörter werden ebenso wie besonders häufige Wörter entfernt.
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, max_df=0.8, min_df=2)

# Berechnen der Vektoren
tfidf_matrix = vectorizer.fit_transform(texts)

# In einem TF-IDF-Embedding hat jeder Absatz einen Vektor, und jede Position im Vektor steht für ein Wort im Vokabular des ganzen Korpus.
# Der Wert für das Wort ist grösser, wenn das Wort in diesem Absatz relativ häufig erscheint.

In diesem Beispiel-Notebook verwenden wir KMeans als Clustering-Algorithmus. Es ist ein schneller Algorithmus, bei dem man die Zahl der gewünschten Cluster im Voraus angibt. Scikit-Learn erklärt die Funktionsweise des Algorithmus in ihrem [User Guide](https://scikit-learn.org/stable/modules/clustering.html#k-means). Eine tolle interaktive Erklärung findet sich auch [hier](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/).

KMeans ist nur ein möglicher Clustering-Algorithmus. Je nach Daten und gewünschten Clustern könnte ein anderer Algorithmus sich besser eignen. Scikit-Learn bietet in seinem User Guide eine gute Übersicht über die üblichen Clustering-Algorithmen.

In [ ]:
from sklearn.cluster import KMeans

# Gewünschte Zahl der Cluster
k = 10
# Vorbereiten des KMeans-Algorithmus
kmeans = KMeans(n_clusters=k, random_state=42)
# Berechnen der Cluster
cluster_labels = kmeans.fit_predict(tfidf_matrix)

# Cluster Labels ist eine Liste mit der gleichen Länge wie die Anzahl der Absätze, auf jeder Position befindet sich eine Zahl für das zugewiesene Cluster.
print(cluster_labels[:10])

In den folgenden zwei Zellen untersuchen wir unsere Cluster. In der ersten Zelle, mit recht komplexem Code, untersuchen wir, welche Wörter pro Cluster besonders relevant sind, um ähnlich wie beim Topic Modeling einen Eindruck über den Inhalt des Clusters zu erhalten.

In [ ]:
# Numpy ist eine Bibliothek, die für komplexe mathematische Operationen grundlegend ist.
import numpy as np

# Wie viele Wörter sollen ausgegeben werden?
top_n = 10
# Anzahl Cluster
num_clusters = k
# Welches Wort befindet sich auf welcher Position in der Tf-idf-Matrix?
feature_names = vectorizer.get_feature_names_out()

top_words_per_cluster = []

for i in range(num_clusters):
    # Sammle alle Vektoren der Absätze, die in diesem Cluster sind
    cluster_indices = np.where(cluster_labels == i)[0]
    if len(cluster_indices) == 0:
        top_words_per_cluster.append([])
        continue

    # Berechne den Durchschnitt der TF-IDF-Werte für alle Absätze in diesem Cluster
    cluster_tfidf = tfidf_matrix[cluster_indices]
    cluster_mean = cluster_tfidf.mean(axis=0)  # mean over all documents in cluster

    # Extrahiere die Top-N Wörter
    cluster_mean_array = np.asarray(cluster_mean).flatten()
    top_indices = np.argsort(cluster_mean_array)[::-1][:top_n]
    top_words = [feature_names[idx] for idx in top_indices]
    
    top_words_per_cluster.append(top_words)

# Die Top-Wörter ausgeben
for i, words in enumerate(top_words_per_cluster):
    print(f"\n--- Cluster {i} ---")
    print(", ".join(words))


In [ ]:
import random

# Zufällige Absätze aus jedem Cluster auswählen
for i in range(num_clusters):
    # Alle Absätze in diesem Cluster
    cluster_indices = np.where(cluster_labels == i)[0]
    if len(cluster_indices) > 0:
        # Suche zufällig einen Absatz aus
        random_index = random.choice(cluster_indices)
        print(f"\n--- Cluster {i} ---")
        print(texts[random_index])

Und was wäre bloss ein Clustering ohne Visualisierung? In unserem Fall eignet sich besonders eine Visualisierung über Zeit, da aber die Metadaten zu den Texten nicht vorhanden sind, nutzen wir einfach den Fakt, dass unsere Liste zeitlich sortiert ist (frühere Ansprachen zuerst). Im ersten Beispiel ist die Vorstellung mit der Bibliothek matplotlib realisiert.

In [ ]:
%pip install -U -q matplotlib pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Wir verwenden pandas um die Daten leichter zu organisieren
df = pd.DataFrame({
    'text': texts,
    'cluster': cluster_labels
})

# Um die Cluster im Zeitverlauf zu analysieren, verwenden wir einen gleitenden Zeitfensteransatz.
window_size = 20  # so viele Absätze in einem Fenster
step = 5  # so weit bewegt sich das Fenster in jedem Schritt

time_bins = []
cluster_counts = []

# Berechnung der Clusterverteilung in jedem Zeitfenster
for start in range(0, len(df) - window_size + 1, step):
    end = start + window_size
    window = df.iloc[start:end]
    counts = window['cluster'].value_counts(normalize=True).sort_index()
    cluster_counts.append(counts)
    time_bins.append(start + window_size // 2)  # midpoint of window

# Leere Fenster mit Null-Werten füllen
cluster_matrix = pd.DataFrame(cluster_counts).fillna(0)

# Den Graph erstellen
plt.figure(figsize=(12, 6))
plt.stackplot(time_bins, cluster_matrix.T.values, labels=[f"Cluster {i}" for i in cluster_matrix.columns])
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1.0))
plt.title("Cluster Trends Over Time")
plt.xlabel("Time (sliding window index)")
plt.ylabel("Proportion of Texts")
plt.tight_layout()
plt.show()


Wow! Hier sehen wir schon gewisse Farbverläufe, die auf eine gewisse Entwicklung der Themen hinweisen. Aber das ganze ist noch etwas unübersichtlich und seltene Cluster sind kaum erkennbar. Um eine interaktive Untersuchung des ganzen zu bewerkstelligen, kann uns die Bibliothek [plotly](https://plotly.com/python/) helfen.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Wir bereiten die Daten wie zuvor auf
df = pd.DataFrame({
    'text': texts,
    'cluster': cluster_labels
})

window_size = 20
step = 5
time_bins = []
cluster_counts = []

for start in range(0, len(df) - window_size + 1, step):
    end = start + window_size
    window = df.iloc[start:end]
    counts = window['cluster'].value_counts(normalize=True).sort_index()
    cluster_counts.append(counts)
    time_bins.append(start + window_size // 2)

cluster_matrix = pd.DataFrame(cluster_counts).fillna(0)

# Dieses Mal erstellen wir also einen interaktiven Graphen
fig = go.Figure()

for cluster in cluster_matrix.columns:
    fig.add_trace(go.Scatter(
        x=time_bins,
        y=cluster_matrix[cluster],
        mode='lines',
        name=f'Cluster {cluster}',
        line=dict(width=2)
    ))

fig.update_layout(
    title='Cluster Trends Over Time (Interactive)',
    xaxis_title='Text Index (Sliding Window Midpoint)',
    yaxis_title='Proportion of Cluster',
    legend_title='Clusters',
    hovermode='x unified',
    template='plotly_white',
    width=1000,
    height=600
)

fig.show()


Das war es erstmal für diese Einführung! Beachte, dass wir diese Auswertung noch an zahlreichen Stellen verbessern könnten:
- Besseres Preprocessing (insbesondere Lemmatisierung)
- Ergänzen der Texte mit Metadaten z.b. zeitlicher Information
- Maskieren von Named Entities, damit sie im Tf-idf die Vektoren weniger verzerren
- Verwendung von anderen Clustering-Algorithmen
- Verwendung von anderen Vektorisierungs-Methoden (z.B. Word2Vec, BERT, etc.)

Viel Spass beim Experimentieren!